# Health Care Bellman

In [ ]:
from collections import namedtuple
import math

State = namedtuple('State', ['period', 'health'])
Action = namedtuple('Action', ['health_inv', 'life_inv'])

def harvest(state):
    return state.health

def end_health(state, action):
    k = 0.01021
    degened_health = state.health - (10 + state.period)
    if degened_health <= 0:
        return 0
    else:
        end_value = 100 * (math.exp(k * action.health_inv) / (math.exp(k * action.health_inv) + ((100 - degened_health) / degened_health)))
        return math.floor(end_value)

def possible_actions(state):
    budget = harvest(state)
    return [Action(health_inv, budget - health_inv) for health_inv in range(budget + 1)]

def immediate_value(state, action):
    c = 464.53
    a = 32
    return c*(end_health(state, action) / 100) * (action.life_inv / (action.life_inv + a))

def transition(state, action):
    return State(state.period + 1, end_health(state, action))


lookup = {}
def opt_health(state):
    if state.period > 10 or state.health <= 0:
        return (0, '')
    if state not in lookup:
        best_val = -1
        best_action = ''
        for action in possible_actions(state):
            opt_val = immediate_value(state, action) + 1*opt_health(transition(state, action))[0]
            if opt_val > best_val:
                best_val = opt_val
                best_action = action
        lookup[state] = (best_val, best_action)
    return lookup[state]

init_state = State(1, 70)
state = init_state
for i in range(10):
    val, dec = opt_health(state)
    print(state, val, dec)
    state = transition(state, dec)


State(period=1, health=70) 987.1731108634837 Action(health_inv=57, life_inv=13)
State(period=2, health=72) 890.5508708634836 Action(health_inv=58, life_inv=14)
State(period=3, health=73) 787.3444230373966 Action(health_inv=58, life_inv=15)
State(period=4, health=73) 679.1188166544179 Action(health_inv=57, life_inv=16)
State(period=5, health=72) 567.6316166544178 Action(health_inv=56, life_inv=16)
State(period=6, health=70) 459.24128332108455 Action(health_inv=54, life_inv=16)
State(period=7, health=67) 355.49624998775124 Action(health_inv=48, life_inv=19)
State(period=8, health=62) 248.19892841912377 Action(health_inv=36, life_inv=26)
State(period=9, health=53) 137.8330077294686 Action(health_inv=13, life_inv=40)
State(period=10, health=37) 42.346285507246385 Action(health_inv=0, life_inv=37)


# Line Wrap Bellman

In [ ]:
from itertools import islice

words = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Morbi pharetra orci quis libero pretium feugiat. Praesent vel viverra mauris, vitae imperdiet eros. Integer semper elementum ligula. Nunc volutpat tortor et aliquet fringilla. Ut vulputate egestas metus, a ullamcorper enim gravida et. Cras massa turpis, consequat sit amet euismod sit amet, feugiat id dui. Nullam ut dolor pellentesque, molestie felis id, volutpat purus. Nullam lacus sem, congue vitae elementum eu, lobortis sed augue. Vestibulum vehicula ultricies diam sed venenatis. Suspendisse blandit dictum est, sit amet pulvinar massa efficitur sed. Proin auctor, velit maximus egestas dictum, enim urna fringilla sapien, efficitur posuere dolor lorem in erat. Sed pulvinar et lectus suscipit blandit.'.split()
width = 70
# words = 'The quick brown fox jumps over the lazy dog'.split()
# words = ['aaa', 'bb', 'cc', 'ddddd']
word_lengths = list(map(len, words))

# width = 14
# width = 6
def needed_space(n, a):
    length = a-1
    for i in range(a):
        length += word_lengths[-n + i]
    return length

def possible_actions(n):
    actions = []
    a = 1
    while a <= n and needed_space(n, a) <= width:
        actions.append(a)
        a += 1
    return actions

def transition(n, a):
    return n-a

def immediate_value(n, a):
    return -(width-needed_space(n, a))**2

def get_best_rag(n):
  action_values = [immediate_value(n,a) + line_wrap(transition(n,a))[0] for a in possible_actions(n)]
  return max(action_values)

def value_action(n, a):
  return (immediate_value(n,a) + line_wrap(transition(n,a))[0], a)

def get_best_rag_action2(n):
  return max(value_action(n,a) for a in possible_actions(n))

def get_best_rag_action(n):
  best_rag, best_action = None, None
  for a in possible_actions(n):
    immediate_rag = immediate_value(n, a)
    future_rag = line_wrap(transition(n, a))[0]
    if not best_action or (immediate_rag + future_rag) > best_rag:
      best_rag, best_action = rag, a
  return best_rag, best_action

lookup = {}
def line_wrap(n):
    if n == 0:
        return (0, '')
    if n not in lookup:
        best_rag, best_action = get_best_rag_action(n)
        lookup[n] = (best_rag, best_action)
    return lookup[n]


# print(possible_actions(3))
# print(needed_space(3, 1))
# print(needed_space(3, 2))
# print(needed_space(3, 3))

print(-line_wrap(len(words))[0])

state = len(words)
lengths = []
while state > 0:
    val, dec = line_wrap(state)
    lengths.append(dec)
    state = transition(state, dec)

it = iter(words)
sliced =[list(islice(it, 0, i)) for i in lengths]
for line in sliced:
    s = ' '.join(line)
    rem_w = width - len(s)
    print(s + ' '*rem_w + '|')


669
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Morbi        |
pharetra orci quis libero pretium feugiat. Praesent vel viverra       |
mauris, vitae imperdiet eros. Integer semper elementum ligula.        |
Nunc volutpat tortor et aliquet fringilla. Ut vulputate egestas       |
metus, a ullamcorper enim gravida et. Cras massa turpis, consequat    |
sit amet euismod sit amet, feugiat id dui. Nullam ut dolor            |
pellentesque, molestie felis id, volutpat purus. Nullam lacus sem,    |
congue vitae elementum eu, lobortis sed augue. Vestibulum vehicula    |
ultricies diam sed venenatis. Suspendisse blandit dictum est, sit     |
amet pulvinar massa efficitur sed. Proin auctor, velit maximus        |
egestas dictum, enim urna fringilla sapien, efficitur posuere         |
dolor lorem in erat. Sed pulvinar et lectus suscipit blandit.         |


In [ ]:
n = len(words)
action_values = [(immediate_value(n,a) + line_wrap(transition(n,a))[0], a) for a in possible_actions(n)]

In [ ]:
action_values

[(-5002, 1),
 (-3587, 2),
 (-2955, 3),
 (-2563, 4),
 (-2055, 5),
 (-1281, 6),
 (-837, 7),
 (-713, 8),
 (-669, 9)]

# Egg Drop Bellman

In [ ]:
from collections import namedtuple

State = namedtuple('State', ['floors', 'eggs'])
def possible_actions(state):
  return range(1, state.floors + 1)

def immediate_value(state, action):
  return 1

# Worst case transition
def transition(state, action):
  survive_state = State(state.floors-action, state.eggs)
  break_state = State(action-1, state.eggs-1)
  if egg_drop(survive_state) > egg_drop(break_state):
    return survive_state
  else:
    return break_state

lookup = {}
def egg_drop(state):
  if state.floors == 0:
    return 0
  if state.eggs == 0:
    return 1000000
  if state not in lookup:
    lookup[state] = min(immediate_value(state, action) + egg_drop(transition(state, action))
                             for action in possible_actions(state))
  return lookup[state]

egg_drop(State(100, 5))


7